In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from ast import literal_eval
from tqdm import tqdm
from sklearn.metrics import classification_report
tqdm.pandas()
from transformers import AutoTokenizer, AutoModel
import random
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
random.seed(0)
from belt_nlp.bert_with_pooling import BertClassifierWithPooling

In [2]:
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'
reports_path = '../reports/'
file_format_users_filtered = processed_data_path + 'r3_{target}_{split}_users_scored_Timeline.csv' 
file_format_tmt_filtered = processed_data_path + '{split}_r3_{target}_top_mentioned_timelines_scored_Texts.csv'

In [3]:
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
    ]

In [4]:

bert_model_name = 'pablocosta/bertabaporu-base-uncased'

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

tokenizer = AutoTokenizer.from_pretrained(bert_model_name, padding=True, truncation=True)


model = AutoModelForSequenceClassification.from_pretrained(
    bert_model_name, num_labels=2, id2label=id2label, label2id=label2id
)

from torchinfo import summary
summary(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              49,152,000
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           1,538
Total params: 13

In [5]:
from sklearn.metrics import f1_score


training_args = TrainingArguments(output_dir='training_dir',
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=3,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=64,
                                  )

def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = 'micro')
  return {'accuracy': acc, 'f1_score': f1}

In [6]:
from datasets import Dataset, DatasetDict

In [7]:
epochs = 3

for target in target_list:
    
    print(f"""
##########################
# Running: {target}
##########################          
          
          """)
    
    # read data
    train_val = pd.read_csv(
        file_format_users_filtered.format(target = target, split = "train"), 
        sep = ';', 
        encoding='utf-8-sig'
        ).reset_index()[['Stance', 'Polarity']].rename(columns = {'Stance': 'text', 'Polarity': 'label'})
    
    train, val = train_test_split(train_val, test_size=0.15, random_state = 42)
    
    train.reset_index(drop=True, inplace=True)
    val.reset_index(drop=True,inplace=True)
    
    
    test = pd.read_csv(
        file_format_users_filtered.format(target = target, split = "test"), 
        sep = ';', 
        encoding='utf-8-sig'
        ).reset_index()[['Stance', 'Polarity']].rename(columns = {'Stance': 'text', 'Polarity': 'label'})
    
    # X_test = test.text.to_numpy()
    # y_test = test.label.map({'against':0, 'for':1}).to_numpy()
    
    # X_train_val = train.text.to_numpy()
    # y_train_val = train.label.map({'against':0, 'for':1}).to_numpy()
    
    # X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state = 42)
    
    tds = Dataset.from_pandas(train)
    testds = Dataset.from_pandas(test)
    vds = Dataset.from_pandas(val)
    dataset = DatasetDict()
    dataset['train'] = tds
    dataset['test'] = testds
    dataset['validation'] = vds
    break


##########################
# Running: ig
##########################          
          
          


In [8]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [9]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/1526 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(bert_model_name).to(device)

In [11]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [12]:
dataset_encoded.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])

In [13]:
dataset_hidden = dataset_encoded.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/1526 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(bert_model_name, num_labels=num_labels)
         .to(device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [16]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = f"{bert_model_name}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

In [17]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["validation"],
                  tokenizer=tokenizer)

trainer.train()

/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/48 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).